In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import pandas as pd
import fiddler as fdl

In [2]:
cassandra.__version__

'3.28.0'

In [3]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'
}

# This token json file is autogenerated when you donwload your token, 
# if yours is different update the file name below
with open("datastax_auth/danny@fiddler.ai-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [4]:
session.set_keyspace('fiddlerai')

In [5]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [39]:
rows = session.execute('SELECT row_id, source_docs, response, question, comment, feedback, ts FROM fiddler_chatbot_history')
    
df = rows._current_rows
df

,row_id,source_docs,response,question,comment,feedback,ts
0,708cbabf-d065-4ff5-8fdf-d72a57edcaef,"Slack\n\nPagerDuty\n\n[block:html]\n{\n ""html...",I could not find an answer.\nJoin our [Slack c...,sadfasdfasdfs,None,NaN,2023-10-11 02:36:53.171
1,8c84be88-c7e6-42fd-83d2-aeefd731ae50,Fiddler provides a Python Client that allows y...,"Yes, Fiddler provides a Python Client that all...",Does Fiddler have a python client?,great answer,1.0,2023-10-11 02:40:42.068
2,fcc7318a-2b9d-4609-ade1-661d35761d70,Ranking is the task of constructing a rank ord...,Ranking is the task of constructing a rank ord...,ranking,great,-1.0,2023-10-11 02:38:38.456


In [16]:
#session.execute("DELETE FROM fiddler_chatbot_conversation WHERE row_id='-1'")

In [35]:
#session.execute("DROP TABLE fiddler_chatbot_history")